In [1]:
import radiomics
from radiomics import featureextractor
import pandas as pd
import os
import SimpleITK as sitk
from concurrent.futures import ThreadPoolExecutor

In [2]:
os.chdir("/media/feike/14Tdisk1/ZHY") 
print(os.getcwd())

/media/feike/14Tdisk1/ZHY


In [3]:
settings = {}
settings['binWidth'] = int(input("请输入binWidth的值：")) # 25
settings['sigma'] = [int(i) for i in input("请输入sigma的值（多个值用空格分隔）：").split()] # 1 2 3
settings['resampledPixelSpacing'] = [int(i) for i in input("请输入resampledPixelSpacing的值（多个值用空格分隔）：").split()] # 1 1 1

请输入binWidth的值： 25
请输入sigma的值（多个值用空格分隔）： 1 2 3
请输入resampledPixelSpacing的值（多个值用空格分隔）： 1 1 1


In [4]:
params = {'binWidth': settings['binWidth'],  'sigma': settings['sigma'], 'resampledPixelSpacing': settings['resampledPixelSpacing']}
params.update(settings)

In [5]:
extractor = featureextractor.RadiomicsFeatureExtractor(**params)
image_types = {}
for imageType in ["Original", "LoG", "Wavelet"]:
    choice = input(f"是否要添加{imageType}特征，输入y/n：").lower()
    if choice == 'y':
        image_types[imageType] = {}

是否要添加Original特征，输入y/n： y
是否要添加LoG特征，输入y/n： y
是否要添加Wavelet特征，输入y/n： y


In [6]:
extractor.enableImageTypes(**image_types)

In [7]:
dataDir = input("请输入数据目录的路径：") # ./CT_first/test

请输入数据目录的路径： ./CT_first/test


In [8]:
# 特征提取

In [9]:
df = pd.DataFrame()
for imageName in os.listdir(os.path.join(dataDir, "images")):
    for maskName in os.listdir(os.path.join(dataDir, "masks")):
        imageFileName = os.path.splitext(imageName)[0]
        maskFileName = os.path.splitext(maskName)[0]
        if imageFileName == maskFileName:
            imagePath = os.path.join(dataDir, "images", imageName)
            maskPath = os.path.join(dataDir, "masks", maskName)
            print(f"正在处理图像文件 {imageFileName} 和掩膜文件 {maskFileName}")
            try:
                featureVector = extractor.execute(imagePath, maskPath)
                df_add = pd.DataFrame.from_dict(featureVector.values()).T
                df_add.columns = featureVector.keys()
                df_add.insert(0, 'imageName', imageFileName)
                df = pd.concat([df, df_add])
            except Exception as e:
                print(f"在处理图像文件 {imageFileName} 和掩膜文件 {maskFileName} 时遇到错误，已跳过这个文件。错误详情: {e}")
                continue

正在处理图像文件 9001247631.nii 和掩膜文件 9001247631.nii
在处理图像文件 9001247631.nii 和掩膜文件 9001247631.nii 时遇到错误，已跳过这个文件。错误详情: Bounding box of ROI is larger than image space:
	ROI bounds (x, y, z image coordinate space) ((153.5, 254.5, -0.5000087278231337), (243.5, 326.5, 107.50187648197372))
	Image Size (512, 512, 108)


In [10]:
df

imageName diagnostics_Versions_PyRadiomics  \
0   0800428919.nii                          3.1.0a2   
0   7100182785.nii                          3.1.0a2   
0   7100191525.nii                          3.1.0a2   
0   7100225999.nii                          3.1.0a2   
0   9001247631.nii                          3.1.0a2   
..             ...                              ...   
0    S07113810.nii                          3.1.0a2   
0    S08238287.nii                          3.1.0a2   
0    S08242489.nii                          3.1.0a2   
0    V01609016.nii                          3.1.0a2   
0    V11533510.nii                          3.1.0a2   

   diagnostics_Versions_Numpy diagnostics_Versions_SimpleITK  \
0                      1.26.4                          2.2.1   
0                      1.26.4                          2.2.1   
0                      1.26.4                          2.2.1   
0                      1.26.4                          2.2.1   
0                      1.26.4                          2.2.1   
..                        ...                            ...   
0                      1.26.4                          2.2.1   
0                      1.26.4                          2.2.1   
0                      1.26.4                          2.2.1   
0                      1.26.4                          2.2.1   
0                      1.26.4                          2.2.1   

   diagnostics_Versions_PyWavelet diagnostics_Versions_Python  \
0                           1.4.1                      3.9.19   
0                           1.4.1                      3.9.19   
0                           1.4.1                      3.9.19   
0                           1.4.1                      3.9.19   
0                           1.4.1                      3.9.19   
..                            ...                         ...   
0                           1.4.1                      3.9.19   
0                           1.4.1                      3.9.19   
0                           1.4.1                      3.9.19   
0                           1.4.1                      3.9.19   
0                           1.4.1                      3.9.19   

                   diagnostics_Configuration_Settings  \
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   
..                                                ...   
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   
0   {'minimumROIDimensions': 2, 'minimumROISize': ...   

   diagnostics_Configuration_EnabledImageTypes  \
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   
..                                         ...   
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   
0   {'Original': {}, 'LoG': {}, 'Wavelet': {}}   

             diagnostics_Image-original_Hash  \
0   d54a83daea9bdca88eadda4e1de9012726bc447a   
0   e1dfd895da01436a0e39179f18730556a7fa4aae   
0   20fa5360dbbd4c564244585b096e42c96114e25c   
0   6730baab00832f866fa441fec2b4c3b123ca514a   
0   908580a33d9285d1e34e18125c354247e4b3d0e7   
..                                       ...   
0   14e293e421413373a2b854e30e3e1b97e55baf33   
0   f15c3ad19df39a21fbc8d017958f2baf3ce3117e   
0   83957a1d9b30c55fc1cabd5a46be14b43d6ece48   
0   98a6f57b969e87e232d45a5ba3a4

In [14]:
result_file = os.path.join(dataDir, 'ml_features_2.xlsx')
df.to_excel(result_file, index=False)
print("结果已保存到文件：", result_file)

结果已保存到文件： ./CT_first/segmentation_nii/ml_features_2.xlsx
